# Medical Image Classification

This notebook demonstrates image classification of medical images using Convolutional Neural Networks (CNNs).
We cover:
- Dataset loading and preprocessing
- Building a CNN model
- Training and evaluation
- Visualization of results


In [ ]:
!pip install torch torchvision matplotlib seaborn scikit-learn pandas numpy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from PIL import Image
import os


In [ ]:
# Sample dataset (images should be in 'data/images/' folder)
data_dir = 'data/images/'

# Sample CSV
df = pd.DataFrame({
    'image_path': ['img1.png','img2.png','img3.png','img4.png','img5.png','img6.png'],
    'label': [0,1,1,0,2,2]  # 0: Negative, 1: Neutral, 2: Positive
})

df.head()


In [ ]:
class MedicalDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_path = os.path.join(data_dir, self.df.iloc[idx]['image_path'])
        image = Image.open(img_path).convert('RGB')
        label = self.df.iloc[idx]['label']
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = MedicalDataset(train_df, transform=transform)
test_dataset = MedicalDataset(test_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2)


In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=3):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3,16,3,padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16,32,3,padding=1)
        self.fc1 = nn.Linear(32*32*32,128)
        self.fc2 = nn.Linear(128,num_classes)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,32*32*32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}')


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100*correct/total:.2f}%')


In [ ]:
# Display some predictions
images, labels = next(iter(test_loader))
outputs = model(images)
_, preds = torch.max(outputs,1)

fig, axs = plt.subplots(1, len(images), figsize=(12,3))
for i in range(len(images)):
    axs[i].imshow(np.transpose(images[i].numpy(), (1,2,0)))
    axs[i].set_title(f'Pred: {preds[i].item()}, True: {labels[i].item()}')
    axs[i].axis('off')
plt.show()


This notebook demonstrates a full pipeline for medical image classification using CNN:
- Dataset loading and preprocessing
- CNN architecture definition
- Model training and evaluation
- Visualization of predictions

The pipeline can be scaled to larger datasets and extended with transfer learning for real-world applications.
